In [1]:
!pip install jupyter-dash dash plotly pandas numpy


In [ ]:
/dev/ADV Project/avg wind speed nwern.xlsx

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from jupyter_dash import JupyterDash as Dash
from dash import dcc, html, Input, Output

# ---------- 1. LOAD DATA (files must be uploaded to Colab first) ----------

harvest = pd.read_csv("/content/h5_i2_2019-2022_hand_harvest.csv")
weather = pd.read_csv("/content/h5-i2_2016-2021_daily-weather.csv")
agb_cc = pd.read_csv("/content/I2_CC_AGB_2020-2022.csv")
massflux = pd.read_csv("/content/Mass Flux values 2019_2022 updated.csv")

# Excel with wind tower averages (Mandan & Morton)
wind_sheets = pd.ExcelFile("/dev/ADV Project/avg wind speed nwern.xlsx")
mandan_daily = pd.read_excel("/dev/ADV Project/avg wind speed nwern.xlsx", sheet_name="MandanDailyAVG")
morton_daily = pd.read_excel("/dev/ADV Project/avg wind speed nwern.xlsx", sheet_name="MortonDailyAVG")

FileNotFoundError: [Errno 2] No such file or directory: '/content/h5_i2_2019-2022_hand_harvest.csv'

In [ ]:
# ---------- 2. CLEAN COLUMN NAMES ----------

def clean_columns(df):
    df = df.copy()
    df.columns = [c.strip().replace(" ", "_") for c in df.columns]
    return df

harvest = clean_columns(harvest)
weather = clean_columns(weather)
agb_cc = clean_columns(agb_cc)
massflux = clean_columns(massflux)
mandan_daily = clean_columns(mandan_daily)
morton_daily = clean_columns(morton_daily)

# ---------- 3. FIX DATES & NUMBERS ----------

# --- Harvest (yield, biomass) ---
harvest["Date"] = pd.to_datetime(harvest["Date"])
for col in ["AGB_g_m2", "Grain_yield_g_m2", "Grain_yield_kg_ha",
            "Stover_g_m2", "Harvest_index", "Percent_H2O"]:
    harvest[col] = pd.to_numeric(harvest[col], errors="coerce")

# --- Cover crop biomass ---
agb_cc["DATE"] = pd.to_datetime(agb_cc["DATE"], errors="coerce")
agb_cc["CC_AGB"] = pd.to_numeric(agb_cc["CC_AGB"], errors="coerce")
agb_cc["Year"] = agb_cc["DATE"].dt.year

# --- Weather (note: FIeld, Date_ / Date_) ---
weather.rename(columns={"FIeld": "Field"}, inplace=True)
# there was a "Date_" with a trailing space originally
if "Date_" in weather.columns:
    weather.rename(columns={"Date_": "Date"}, inplace=True)
if "Date" not in weather.columns and "Date_" in weather.columns:
    weather.rename(columns={"Date_": "Date"}, inplace=True)

weather["Date"] = pd.to_datetime(weather["Date"].astype(str), format="%Y-%m-%d")
for col in ["Sol_Rad_MJ_m2_d", "T_min_C", "T_max_C",
            "PCPN_mm_d", "RH_f", "Wind_spd_m_s"]:
    weather[col] = pd.to_numeric(weather[col], errors="coerce")

# --- Mass flux / wind erosion ---
massflux["Year"] = pd.to_numeric(massflux["Year"], errors="coerce")
massflux["Date"] = massflux["Date"].astype(str).str.zfill(4)
massflux["Month"] = massflux["Date"].str[:2].astype(int)
massflux["Day"] = massflux["Date"].str[2:].astype(int)

massflux["Date_full"] = pd.to_datetime(
    dict(year=massflux["Year"], month=massflux["Month"], day=massflux["Day"]),
    errors="coerce"
)

# column names are "Days.After.Planting" and "total.flux" after cleaning
massflux.rename(columns={"total.flux": "total_flux"}, inplace=True)
massflux["total_flux"] = pd.to_numeric(massflux["total_flux"], errors="coerce")
if "Days.After.Planting" in massflux.columns:
    massflux["Days_After_Planting"] = pd.to_numeric(
        massflux["Days.After.Planting"], errors="coerce"
    )

# --- Wind tower daily average speed ---
for df, site_name in [(mandan_daily, "Mandan"), (morton_daily, "Morton")]:
    df["date"] = pd.to_datetime(df["date"])
    # col is "Average_of_AvgWS6_10M_m/s" after cleaning
    if "Average_of_AvgWS6_10M_m/s" in df.columns:
        df.rename(columns={"Average_of_AvgWS6_10M_m/s": "wind_10m_m_s"},
                  inplace=True)
    df["Site"] = site_name

wind_daily = pd.concat([mandan_daily, morton_daily], ignore_index=True)

# ---------- 4. AGGREGATED TABLES FOR THEMES ----------

# Theme 1 – yield & biomass
harvest_theme = (harvest
                 .groupby(["Year", "Crop", "Treatment"], as_index=False)
                 .agg(mean_yield=("Grain_yield_kg_ha", "mean"),
                      mean_agb=("AGB_g_m2", "mean"),
                      mean_hi=("Harvest_index", "mean")))

# Theme 2 – erosion by site/treatment
erosion_theme = (massflux
                 .groupby(["Site", "Treatment"], as_index=False)
                 .agg(mean_flux=("total_flux", "mean"),
                      max_flux=("total_flux", "max")))

# Theme 3 – weather summaries
weather_theme = (weather
                 .groupby(["Year", "Field"], as_index=False)
                 .agg(mean_wind=("Wind_spd_m_s", "mean"),
                      mean_pcpn=("PCPN_mm_d", "mean"),
                      mean_tmin=("T_min_C", "mean"),
                      mean_tmax=("T_max_C", "mean")))

# Theme 4 – trends
yield_by_year = (harvest
                 .groupby(["Year", "Treatment"], as_index=False)
                 .agg(mean_yield=("Grain_yield_kg_ha", "mean")))

flux_by_year = (massflux
                .groupby(["Year", "Site", "Treatment"], as_index=False)
                .agg(mean_flux=("total_flux", "mean")))

# Theme 5 – treatment performance
treatment_perf = (harvest
                  .groupby("Treatment", as_index=False)
                  .agg(mean_yield=("Grain_yield_kg_ha", "mean"),
                       mean_agb=("AGB_g_m2", "mean"),
                       mean_hi=("Harvest_index", "mean")))

cc_perf = (agb_cc
           .groupby(["TRT", "Year"], as_index=False)
           .agg(mean_cc_agb=("CC_AGB", "mean")))

In [ ]:
# ---------- THEME 1: Yield & Biomass ----------

def fig_yield_by_crop_treatment(harvest_theme, year=None):
    df = harvest_theme.copy()
    if year is not None:
        df = df[df["Year"] == year]

    fig = px.bar(
        df,
        x="Crop",
        y="mean_yield",
        color="Treatment",
        barmode="group",
        facet_col="Year" if year is None else None,
        title="Grain yield (kg/ha) by crop and treatment",
        labels={"mean_yield": "Mean grain yield (kg/ha)"}
    )
    fig.update_layout(legend_title_text="Treatment")
    return fig


def fig_agb_vs_yield_scatter(harvest_theme):
    fig = px.scatter(
        harvest_theme,
        x="mean_agb",
        y="mean_yield",
        color="Treatment",
        symbol="Crop",
        size="mean_hi",
        hover_data=["Year", "Crop"],
        title="Biomass vs grain yield (marker size = harvest index)"
    )
    fig.update_layout(
        xaxis_title="Aboveground biomass (g/m²)",
        yaxis_title="Grain yield (kg/ha)"
    )
    return fig


# ---------- THEME 2: Wind Erosion & Mass Flux ----------

def fig_flux_by_site_treatment(erosion_theme):
    fig = px.bar(
        erosion_theme,
        x="Site",
        y="mean_flux",
        color="Treatment",
        barmode="group",
        title="Average total mass flux by site and treatment",
        labels={"mean_flux": "Mean total flux (g m⁻¹ d⁻¹)"}
    )
    return fig


def fig_flux_time_series(massflux, site=None, treatment=None):
    df = massflux.dropna(subset=["Date_full", "total_flux"]).copy()
    if site:
        df = df[df["Site"] == site]
    if treatment:
        df = df[df["Treatment"] == treatment]

    fig = px.line(
        df,
        x="Date_full",
        y="total_flux",
        color="Location",
        title="Wind erosion mass flux over time",
        labels={"total_flux": "Total flux (g m⁻¹ d⁻¹)", "Date_full": "Date"}
    )
    return fig


# ---------- THEME 3: Weather Impacts ----------

def fig_weather_by_field(weather_theme):
    fig = px.bar(
        weather_theme,
        x="Year",
        y="mean_wind",
        color="Field",
        barmode="group",
        title="Average wind speed by field and year",
        labels={"mean_wind": "Mean daily wind speed (m/s)"}
    )
    return fig


def fig_precip_vs_flux(weather, massflux):
    wf = weather[["Date", "Year", "PCPN_mm_d", "Wind_spd_m_s"]].copy()
    mf = massflux[["Date_full", "Year", "Site", "Treatment", "total_flux"]].copy()

    merged = pd.merge(
        mf,
        wf,
        left_on=["Year", "Date_full"],
        right_on=["Year", "Date"],
        how="left"
    ).dropna(subset=["total_flux"])

    fig = px.scatter(
        merged,
        x="PCPN_mm_d",
        y="total_flux",
        color="Site",
        trendline="ols",
        title="Daily precipitation vs wind erosion flux",
        labels={"PCPN_mm_d": "Precipitation (mm/day)",
                "total_flux": "Total flux (g m⁻¹ d⁻¹)"}
    )
    return fig


def fig_wind_vs_flux(weather, massflux):
    wf = weather[["Date", "Year", "Wind_spd_m_s"]].copy()
    mf = massflux[["Date_full", "Year", "Site", "Treatment", "total_flux"]].copy()

    merged = pd.merge(
        mf,
        wf,
        left_on=["Year", "Date_full"],
        right_on=["Year", "Date"],
        how="left"
    ).dropna(subset=["total_flux", "Wind_spd_m_s"])

    fig = px.scatter(
        merged,
        x="Wind_spd_m_s",
        y="total_flux",
        color="Site",
        trendline="ols",
        title="Wind speed vs wind erosion flux",
        labels={"Wind_spd_m_s": "Wind speed (m/s)",
                "total_flux": "Total flux (g m⁻¹ d⁻¹)"}
    )
    return fig


# ---------- THEME 4: Time Trends & Seasonality ----------

def fig_yield_trend(yield_by_year):
    fig = px.line(
        yield_by_year,
        x="Year",
        y="mean_yield",
        color="Treatment",
        markers=True,
        title="Trend in mean grain yield over time",
        labels={"mean_yield": "Mean grain yield (kg/ha)"}
    )
    return fig


def fig_flux_trend(flux_by_year):
    fig = px.line(
        flux_by_year,
        x="Year",
        y="mean_flux",
        color="Site",
        line_dash="Treatment",
        markers=True,
        title="Trend in mean wind erosion over time",
        labels={"mean_flux": "Mean total flux (g m⁻¹ d⁻¹)"}
    )
    return fig


def fig_erosion_seasonality(massflux):
    df = massflux.dropna(subset=["Date_full", "total_flux"]).copy()
    df["Month"] = df["Date_full"].dt.month
    seasonality = (df
                   .groupby(["Month", "Site", "Treatment"], as_index=False)
                   .agg(mean_flux=("total_flux", "mean")))
    fig = px.line(
        seasonality,
        x="Month",
        y="mean_flux",
        color="Site",
        line_dash="Treatment",
        markers=True,
        title="Seasonal pattern of wind erosion by month",
        labels={"mean_flux": "Mean total flux (g m⁻¹ d⁻¹)"}
    )
    return fig


# ---------- THEME 5: Treatment Performance ----------

def fig_treatment_overall_yield(treatment_perf):
    fig = px.bar(
        treatment_perf,
        x="Treatment",
        y="mean_yield",
        color="Treatment",
        title="Overall mean grain yield by treatment",
        labels={"mean_yield": "Mean grain yield (kg/ha)"}
    )
    return fig


def fig_treatment_cc_biomass(cc_perf):
    fig = px.bar(
        cc_perf,
        x="Year",
        y="mean_cc_agb",
        color="TRT",
        barmode="group",
        title="Cover crop AGB by year and treatment",
        labels={"mean_cc_agb": "Mean CC AGB (g/m²)"}
    )
    return fig


def fig_yield_by_crop_treatment_rank(harvest_theme):
    df = harvest_theme.copy()
    df["label"] = df["Crop"] + " " + df["Treatment"] + " " + df["Year"].astype(str)
    df = df.sort_values("mean_yield", ascending=False)

    fig = px.bar(
        df,
        x="label",
        y="mean_yield",
        title="Ranking of crop–treatment–year by mean yield",
        labels={"mean_yield": "Mean grain yield (kg/ha)", "label": "Crop–Treatment–Year"}
    )
    fig.update_xaxes(tickangle=45)
    return fig

In [ ]:
# ---------- DASH APP LAYOUT ----------

app = Dash(__name__)

available_years = sorted(harvest["Year"].dropna().unique())
available_sites = sorted(massflux["Site"].dropna().unique())
available_treatments = sorted(harvest["Treatment"].dropna().unique())

app.layout = html.Div([
    html.H1("Crop Yield & Wind Erosion – Conservation Practice Dashboard"),

    # KPI row
    html.Div([
        html.Div([
            html.H3("Avg grain yield (kg/ha)"),
            html.H2(f"{harvest['Grain_yield_kg_ha'].mean():.0f}")
        ], style={"padding": "10px", "border": "1px solid #ddd"}),

        html.Div([
            html.H3("Avg mass flux (g m⁻¹ d⁻¹)"),
            html.H2(f"{massflux['total_flux'].mean():.2f}")
        ], style={"padding": "10px", "border": "1px solid #ddd"}),

        html.Div([
            html.H3("Avg wind speed (m/s)"),
            html.H2(f"{weather['Wind_spd_m_s'].mean():.2f}")
        ], style={"padding": "10px", "border": "1px solid #ddd"})
    ], style={"display": "flex", "gap": "1rem"}),

    dcc.Tabs([
        dcc.Tab(label="Theme 1 – Yield & Biomass", children=[
            html.Div([
                html.Label("Select year (optional):"),
                dcc.Dropdown(
                    options=[{"label": str(y), "value": int(y)} for y in available_years],
                    value=None,
                    id="theme1-year"
                ),
                dcc.Graph(id="theme1-yield-bar"),
                dcc.Graph(id="theme1-agb-scatter")
            ])
        ]),

        dcc.Tab(label="Theme 2 – Wind Erosion", children=[
            html.Div([
                html.Label("Site:"),
                dcc.Dropdown(
                    options=[{"label": s, "value": s} for s in available_sites],
                    value=None,
                    id="theme2-site"
                ),
                html.Label("Treatment:"),
                dcc.Dropdown(
                    options=[{"label": t, "value": t} for t in available_treatments],
                    value=None,
                    id="theme2-treatment"
                ),
                dcc.Graph(id="theme2-site-bar"),
                dcc.Graph(id="theme2-flux-ts")
            ])
        ]),

        dcc.Tab(label="Theme 3 – Weather & Environment", children=[
            html.Div([
                dcc.Graph(id="theme3-weather-field"),
                dcc.Graph(id="theme3-precip-flux"),
                dcc.Graph(id="theme3-wind-flux")
            ])
        ]),

        dcc.Tab(label="Theme 4 – Time Trends", children=[
            html.Div([
                dcc.Graph(id="theme4-yield-trend"),
                dcc.Graph(id="theme4-flux-trend"),
                dcc.Graph(id="theme4-seasonality")
            ])
        ]),

        dcc.Tab(label="Theme 5 – Treatment Performance", children=[
            html.Div([
                dcc.Graph(id="theme5-overall-yield"),
                dcc.Graph(id="theme5-cc-biomass"),
                dcc.Graph(id="theme5-yield-rank")
            ])
        ])
    ])
])

# ---------- CALLBACKS ----------

@app.callback(
    Output("theme1-yield-bar", "figure"),
    Output("theme1-agb-scatter", "figure"),
    Input("theme1-year", "value")
)
def update_theme1(year):
    fig1 = fig_yield_by_crop_treatment(harvest_theme, year)
    fig2 = fig_agb_vs_yield_scatter(harvest_theme)
    return fig1, fig2


@app.callback(
    Output("theme2-site-bar", "figure"),
    Output("theme2-flux-ts", "figure"),
    Input("theme2-site", "value"),
    Input("theme2-treatment", "value")
)
def update_theme2(site, treatment):
    fig1 = fig_flux_by_site_treatment(erosion_theme)
    fig2 = fig_flux_time_series(massflux, site=site, treatment=treatment)
    return fig1, fig2


@app.callback(
    Output("theme3-weather-field", "figure"),
    Output("theme3-precip-flux", "figure"),
    Output("theme3-wind-flux", "figure"),
    Input("theme2-site", "value")  # just to trigger updates
)
def update_theme3(_site):
    fig1 = fig_weather_by_field(weather_theme)
    fig2 = fig_precip_vs_flux(weather, massflux)
    fig3 = fig_wind_vs_flux(weather, massflux)
    return fig1, fig2, fig3


@app.callback(
    Output("theme4-yield-trend", "figure"),
    Output("theme4-flux-trend", "figure"),
    Output("theme4-seasonality", "figure"),
    Input("theme1-year", "value")  # trigger
)
def update_theme4(_):
    return (
        fig_yield_trend(yield_by_year),
        fig_flux_trend(flux_by_year),
        fig_erosion_seasonality(massflux)
    )


@app.callback(
    Output("theme5-overall-yield", "figure"),
    Output("theme5-cc-biomass", "figure"),
    Output("theme5-yield-rank", "figure"),
    Input("theme1-year", "value")  # trigger
)
def update_theme5(_):
    return (
        fig_treatment_overall_yield(treatment_perf),
        fig_treatment_cc_biomass(cc_perf),
        fig_yield_by_crop_treatment_rank(harvest_theme)
    )


In [ ]:
!pip install plotly pandas numpy


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# === LOAD DATA (filenames must match your uploads) ===
harvest   = pd.read_csv("h5_i2_2019-2022_hand_harvest.csv")
weather   = pd.read_csv("h5-i2_2016-2021_daily-weather.csv")
agb_cc    = pd.read_csv("I2_CC_AGB_2020-2022.csv")
massflux  = pd.read_csv("Mass Flux values 2019_2022 updated.csv")
wind_xls  = "/dev/ADV Project/avg wind speed nwern.xlsx"   # Excel with wind tower info

mandan_daily = pd.read_excel(wind_xls, sheet_name="MandanDailyAVG")
morton_daily = pd.read_excel(wind_xls, sheet_name="MortonDailyAVG")

print("Loaded:")
print("harvest:", harvest.shape)
print("weather:", weather.shape)
print("agb_cc:", agb_cc.shape)
print("massflux:", massflux.shape)
print("mandan_daily:", mandan_daily.shape, "morton_daily:", morton_daily.shape)

Loaded:
harvest: (260, 15)
weather: (4384, 12)
agb_cc: (31, 6)
massflux: (563, 13)
mandan_daily: (242, 2) morton_daily: (240, 2)


In [ ]:
# ---------- BASIC CLEANING ----------

def clean_cols(df):
    df = df.copy()
    df.columns = [c.strip().replace(" ", "_") for c in df.columns]
    return df

harvest      = clean_cols(harvest)
weather      = clean_cols(weather)
agb_cc       = clean_cols(agb_cc)
massflux     = clean_cols(massflux)
mandan_daily = clean_cols(mandan_daily)
morton_daily = clean_cols(morton_daily)

# === Harvest data ===
harvest["Date"] = pd.to_datetime(harvest["Date"], errors="coerce")
for col in ["AGB_g_m2", "Grain_yield_g_m2", "Grain_yield_kg_ha",
            "Stover_g_m2", "Harvest_index", "Percent_H2O"]:
    if col in harvest.columns:
        harvest[col] = pd.to_numeric(harvest[col], errors="coerce")

# ensure Year exists
if "Year" not in harvest.columns:
    harvest["Year"] = harvest["Date"].dt.year

# === Cover crop biomass ===
agb_cc["DATE"] = pd.to_datetime(agb_cc["DATE"], errors="coerce")
agb_cc["Year"] = agb_cc["DATE"].dt.year
agb_cc["CC_AGB"] = pd.to_numeric(agb_cc["CC_AGB"], errors="coerce")

# === Weather ===
# fix FIeld typo and Date column
if "FIeld" in weather.columns and "Field" not in weather.columns:
    weather.rename(columns={"FIeld": "Field"}, inplace=True)

if "Date_" in weather.columns and "Date" not in weather.columns:
    weather.rename(columns={"Date_": "Date"}, inplace=True)

weather["Date"] = pd.to_datetime(weather["Date"].astype(str), format="%Y-%m-%d")
for col in ["Sol_Rad_MJ_m2_d", "T_min_C", "T_max_C",
            "PCPN_mm_d", "RH_f", "Wind_spd_m_s"]:
    if col in weather.columns:
        weather[col] = pd.to_numeric(weather[col], errors="coerce")

weather["Year"] = weather["Date"].dt.year

# === Mass flux / erosion ===
massflux["Year"] = pd.to_numeric(massflux["Year"], errors="coerce")

# find the total-flux column robustly
flux_col = None
for c in massflux.columns:
    if "total" in c.lower() and "flux" in c.lower():
        flux_col = c
        break

if flux_col is None:
    raise ValueError("Cannot find total flux column in massflux; check column names.")

massflux.rename(columns={flux_col: "total_flux"}, inplace=True)
massflux["total_flux"] = pd.to_numeric(massflux["total_flux"], errors="coerce")

# Date is typically numeric like 613 -> June 13
massflux["Date"] = massflux["Date"].astype(str).str.zfill(4)
massflux["Month"] = massflux["Date"].str[:2].astype(int)
massflux["Day"]   = massflux["Date"].str[2:].astype(int)

massflux["Date_full"] = pd.to_datetime(
    dict(year=massflux["Year"], month=massflux["Month"], day=massflux["Day"]),
    errors="coerce"
)

if "Days.After.Planting" in massflux.columns:
    massflux["Days_After_Planting"] = pd.to_numeric(
        massflux["Days.After.Planting"], errors="coerce"
    )

# === Wind tower daily data ===
for df, site_name in [(mandan_daily, "Mandan"), (morton_daily, "Morton")]:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    # find the avg wind speed column
    wcol = None
    for c in df.columns:
        if "avgws6" in c.lower() or ("10m" in c.lower() and "avg" in c.lower()):
            wcol = c
            break
    if wcol is not None:
        df.rename(columns={wcol: "wind_10m_m_s"}, inplace=True)
    df["Site"] = site_name

wind_daily = pd.concat([mandan_daily, morton_daily], ignore_index=True)

# ---------- AGGREGATED TABLES PER THEME ----------

# Theme 1 – yield & biomass
harvest_theme = (harvest
                 .groupby(["Year", "Crop", "Treatment"], as_index=False)
                 .agg(mean_yield=("Grain_yield_kg_ha", "mean"),
                      mean_agb=("AGB_g_m2", "mean"),
                      mean_hi=("Harvest_index", "mean")))

# Theme 2 – erosion by site/treatment
erosion_theme = (massflux
                 .groupby(["Site", "Treatment"], as_index=False)
                 .agg(mean_flux=("total_flux", "mean"),
                      max_flux=("total_flux", "max")))

# Theme 3 – weather summaries
weather_theme = (weather
                 .groupby(["Year", "Field"], as_index=False)
                 .agg(mean_wind=("Wind_spd_m_s", "mean"),
                      mean_pcpn=("PCPN_mm_d", "mean"),
                      mean_tmin=("T_min_C", "mean"),
                      mean_tmax=("T_max_C", "mean")))

# Theme 4 – trends
yield_by_year = (harvest
                 .groupby(["Year", "Treatment"], as_index=False)
                 .agg(mean_yield=("Grain_yield_kg_ha", "mean")))

flux_by_year = (massflux
                .groupby(["Year", "Site", "Treatment"], as_index=False)
                .agg(mean_flux=("total_flux", "mean")))

# Theme 5 – treatment performance & cover crops
treatment_perf = (harvest
                  .groupby("Treatment", as_index=False)
                  .agg(mean_yield=("Grain_yield_kg_ha", "mean"),
                       mean_agb=("AGB_g_m2", "mean"),
                       mean_hi=("Harvest_index", "mean")))

cc_perf = (agb_cc
           .groupby(["TRT", "Year"], as_index=False)
           .agg(mean_cc_agb=("CC_AGB", "mean")))

In [ ]:
# Bar: mean yield by crop, treatment, year
fig_t1_yield = px.bar(
    harvest_theme,
    x="Crop",
    y="mean_yield",
    color="Treatment",
    barmode="group",
    facet_col="Year",
    title="Theme 1: Grain yield (kg/ha) by crop and treatment over years",
    labels={"mean_yield": "Mean grain yield (kg/ha)"}
)
fig_t1_yield.show()

# Scatter: biomass vs yield (size = harvest index)
fig_t1_agb_vs_yield = px.scatter(
    harvest_theme,
    x="mean_agb",
    y="mean_yield",
    color="Treatment",
    symbol="Crop",
    size="mean_hi",
    hover_data=["Year", "Crop"],
    title="Theme 1: Biomass vs grain yield (size = harvest index)",
)
fig_t1_agb_vs_yield.update_layout(
    xaxis_title="Aboveground biomass (g/m²)",
    yaxis_title="Grain yield (kg/ha)"
)
fig_t1_agb_vs_yield.show()


In [ ]:
# Bar: mean flux by site & treatment
fig_t2_flux = px.bar(
    erosion_theme,
    x="Site",
    y="mean_flux",
    color="Treatment",
    barmode="group",
    title="Theme 2: Average total mass flux by site and treatment",
    labels={"mean_flux": "Mean total flux (g m⁻¹ d⁻¹)"}
)
fig_t2_flux.show()

# Time series: daily flux over time (all sites & treatments)
fig_t2_ts = px.line(
    massflux.dropna(subset=["Date_full", "total_flux"]),
    x="Date_full",
    y="total_flux",
    color="Site",
    line_group="Location" if "Location" in massflux.columns else None,
    title="Theme 2: Wind erosion mass flux over time",
    labels={"total_flux": "Total flux (g m⁻¹ d⁻¹)", "Date_full": "Date"}
)
fig_t2_ts.show()


In [ ]:
# Bar: mean wind by field & year
fig_t3_wind = px.bar(
    weather_theme,
    x="Year",
    y="mean_wind",
    color="Field",
    barmode="group",
    title="Theme 3: Average wind speed by field and year",
    labels={"mean_wind": "Mean daily wind speed (m/s)"}
)
fig_t3_wind.show()

# Merge weather & flux by (Year, date)
wf = weather[["Date", "Year", "PCPN_mm_d", "Wind_spd_m_s"]].copy()
mf = massflux[["Date_full", "Year", "Site", "Treatment", "total_flux"]].copy()

merged = pd.merge(
    mf,
    wf,
    left_on=["Year", "Date_full"],
    right_on=["Year", "Date"],
    how="left"
).dropna(subset=["total_flux"])

# Scatter: precipitation vs flux
fig_t3_pcpn_flux = px.scatter(
    merged,
    x="PCPN_mm_d",
    y="total_flux",
    color="Site",
    trendline="ols",
    title="Theme 3: Daily precipitation vs wind erosion",
    labels={"PCPN_mm_d": "Precipitation (mm/day)",
            "total_flux": "Total flux (g m⁻¹ d⁻¹)"}
)
fig_t3_pcpn_flux.show()

# Scatter: wind speed vs flux
fig_t3_wind_flux = px.scatter(
    merged,
    x="Wind_spd_m_s",
    y="total_flux",
    color="Site",
    trendline="ols",
    title="Theme 3: Wind speed vs wind erosion",
    labels={"Wind_spd_m_s": "Wind speed (m/s)",
            "total_flux": "Total flux (g m⁻¹ d⁻¹)"}
)
fig_t3_wind_flux.show()


In [ ]:
# Yield trend over years by treatment
fig_t4_yield_trend = px.line(
    yield_by_year,
    x="Year",
    y="mean_yield",
    color="Treatment",
    markers=True,
    title="Theme 4: Trend in mean grain yield over time",
    labels={"mean_yield": "Mean grain yield (kg/ha)"}
)
fig_t4_yield_trend.show()

# Flux trend over years by site & treatment
fig_t4_flux_trend = px.line(
    flux_by_year,
    x="Year",
    y="mean_flux",
    color="Site",
    line_dash="Treatment",
    markers=True,
    title="Theme 4: Trend in wind erosion over time",
    labels={"mean_flux": "Mean total flux (g m⁻¹ d⁻¹)"}
)
fig_t4_flux_trend.show()

# Seasonality: avg monthly flux
flux_month = massflux.dropna(subset=["Date_full", "total_flux"]).copy()
flux_month["Month"] = flux_month["Date_full"].dt.month

seasonality = (flux_month
               .groupby(["Month", "Site", "Treatment"], as_index=False)
               .agg(mean_flux=("total_flux", "mean")))

fig_t4_seasonality = px.line(
    seasonality,
    x="Month",
    y="mean_flux",
    color="Site",
    line_dash="Treatment",
    markers=True,
    title="Theme 4: Seasonal pattern of wind erosion (by month)",
    labels={"mean_flux": "Mean total flux (g m⁻¹ d⁻¹)"}
)
fig_t4_seasonality.show()


In [ ]:
# Overall yield by treatment
fig_t5_yield_treatment = px.bar(
    treatment_perf,
    x="Treatment",
    y="mean_yield",
    color="Treatment",
    title="Theme 5: Overall mean grain yield by treatment",
    labels={"mean_yield": "Mean grain yield (kg/ha)"}
)
fig_t5_yield_treatment.show()

# Cover crop biomass by year and treatment
fig_t5_cc_biomass = px.bar(
    cc_perf,
    x="Year",
    y="mean_cc_agb",
    color="TRT",
    barmode="group",
    title="Theme 5: Cover crop AGB by year and treatment",
    labels={"mean_cc_agb": "Mean CC AGB (g/m²)"}
)
fig_t5_cc_biomass.show()

# Ranking of crop–treatment–year by yield
harvest_theme_rank = harvest_theme.copy()
harvest_theme_rank["label"] = (
    harvest_theme_rank["Crop"] + " " +
    harvest_theme_rank["Treatment"] + " " +
    harvest_theme_rank["Year"].astype(str)
)
harvest_theme_rank = harvest_theme_rank.sort_values("mean_yield", ascending=False)

fig_t5_rank = px.bar(
    harvest_theme_rank,
    x="label",
    y="mean_yield",
    title="Theme 5: Ranking of crop–treatment–year by mean yield",
    labels={"mean_yield": "Mean grain yield (kg/ha)",
            "label": "Crop–Treatment–Year"}
)
fig_t5_rank.update_xaxes(tickangle=45)
fig_t5_rank.show()
